In [ ]:
#!pip install schedule
#!pip install snowflake-connector-python

In [ ]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time


# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)
# Email credentials 
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb' 
receiver_email = 'abhitiwari.sirka2001@kgpian.iitkgp.ac.in'
query=None
def transform(query):
    # Execute Snowflake SQL Queries
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")


    result_df = pd.read_sql_query(query, conn)
    
    return result_df


def fetch_transform_send():

    # Replace 'your_table' with the actual table name
    query = f"SHOW TABLES IN SCHEMA {schema}"
    result_df = transform(query)

    # Save DataFrame to an Excel file
    excel_file_path = 'transformed_data.csv'
    result_df.to_csv(excel_file_path, index=False)

    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data.csv')

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = receiver_email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, receiver_email, email_message.as_string())

    conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)



In [ ]:
# Run the scheduler loop
while True:
    schedule.run_pending()
    time.sleep(60)


Final Code for scheduler

CURRENTLY WORKING CELL

In [1]:
import os
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_dir = 'C:\\Users\\User\\Downloads\\Eggoz\\Customer_Segmentation'
excel_file_name = 'transformed_data_output_3.xlsx'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )

# Construct the full path to the Excel file
excel_file_path = os.path.join(output_dir, excel_file_name)

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')

def transform(query):
    # Execute Snowflake SQL Queries
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email_with_attachment(name, email):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename=excel_file_name)

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():
    queries = [
        """
        SELECT ID, NAME
        FROM eggozdb.maplemonk.my_sql_product_product
        """,
        """
        SELECT ID, CODE
        FROM eggozdb.maplemonk.my_sql_retailer_retailer
        """,
        """
        SELECT DATE, SKU_ORDER_PRICE_AMOUNT
        FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
        """
    ]

    sheet_names = ['Sheet1', 'Sheet2', 'Sheet3']

    # Create a dictionary to store dataframes for each sheet
    dataframes = {}

    # Iterate over queries and store dataframes in the dictionary
    for query, sheet_name in zip(queries, sheet_names):
        dataframes[sheet_name] = transform(query)

    # Write all dataframes to the Excel file
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        for sheet_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Send customized emails to each recipient
    for index, row in emails_df.iterrows():
        send_email_with_attachment(row['name'], row['email'])
        if index == len(emails_df) - 1:
            break

    conn.close()
# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)

Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-25 13:20:27)

In [2]:
while True:
    schedule.run_pending()
    time.sleep(60)
    

C:\Users\User\AppData\Local\Temp\ipykernel_17700\1467579802.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_17700\1467579802.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_17700\1467579802.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)


KeyboardInterrupt: 

In [4]:
import os
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time
import json

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_dir = 'C:\\Users\\User\\Downloads\\Eggoz\\Customer_Segmentation'
excel_file_name = 'transformed_data_output_3.xlsx'
config_file_path = 'config.json'  # Path to the configuration file

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )

# Construct the full path to the Excel file
excel_file_path = os.path.join(output_dir, excel_file_name)

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')

# change the query and sheet name accordingly
config={
    "queries": [
        {
            "query": "SELECT ID, NAME FROM eggozdb.maplemonk.my_sql_product_product",
            "sheet_name": "Sheet1_akt"
        },
        {
            "query": "SELECT ID, CODE FROM eggozdb.maplemonk.my_sql_retailer_retailer",
            "sheet_name": "Sheet2_sec"
        },
        {
            "query": "SELECT DATE, SKU_ORDER_PRICE_AMOUNT FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary",
            "sheet_name": "Sheet3_eggoz"
        }
    ]
}

def transform(query):
    # Execute Snowflake SQL Queries
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )

    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email_with_attachment(name, email):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename=excel_file_name)

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():


    # Create a dictionary to store dataframes for each sheet
    dataframes = {}

    # Iterate over queries and store dataframes in the dictionary
    for query_info in config['queries']:
        query = query_info['query']
        sheet_name = query_info['sheet_name']
        dataframes[sheet_name] = transform(query)

    # Write all dataframes to the Excel file
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        for sheet_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Send customized emails to each recipient
    for index, row in emails_df.iterrows():
        send_email_with_attachment(row['name'], row['email'])
        if index == len(emails_df) - 1:
            break

    # Close the Snowflake connection
    conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)



Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-25 13:45:07)

In [2]:
def __main__():
    fetch_transform_send()

In [5]:

# Run the schedule
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_19160\758216300.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_19160\758216300.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_19160\758216300.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_19160\758216300.py:75

In [4]:
import os
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import json

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_dir = 'C:\\Users\\User\\Downloads\\Eggoz\\Customer_Segmentation'
config_file_path = 'config.json'  # Path to the configuration file

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def transform(query):
    # Execute Snowflake SQL Queries
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )

    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email_with_attachment(name, email, excel_file_path):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_file_path))

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send(queries, emails_df):
    # Create a dictionary to store dataframes for each sheet
    dataframes = {}

    # Iterate over queries and store dataframes in the dictionary
    for query_info in queries:
        query = query_info['query']
        sheet_name = query_info['sheet_name']
        dataframes[sheet_name] = transform(query)

    # Create a unique identifier for the Excel file (e.g., timestamp)
    timestamp = pd.to_datetime('now').strftime('%Y%m%d%H%M%S')
    excel_file_name = f'transformed_data_{timestamp}.xlsx'
    excel_file_path = os.path.join(output_dir, excel_file_name)

    # Write all dataframes to the Excel file
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        for sheet_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Send customized emails to each recipient
    for index, row in emails_df.iterrows():
        send_email_with_attachment(row['name'], row['email'], excel_file_path)

# Example queries
example_queries = [
    {
        "query": "SELECT ID, NAME FROM eggozdb.maplemonk.my_sql_product_product",#change the query
        "sheet_name": "Custom_sheet_1" # customize the name of the sheet
    },
    {
        "query": "SELECT ID, CODE FROM eggozdb.maplemonk.my_sql_retailer_retailer", #change the query
        "sheet_name": "Custom_sheet_2" # customize the name of the sheet
    },
    {
        "query": "SELECT DATE, SKU_ORDER_PRICE_AMOUNT FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary", #change the query
        "sheet_name": "Custom_sheet_3" # customize the name of the sheet
    }
]

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')

# Use the function to fetch, transform, and send the data
fetch_transform_send(example_queries, emails_df)


C:\Users\User\AppData\Local\Temp\ipykernel_12092\509317492.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12092\509317492.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12092\509317492.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12092\509317492.py:90

KeyboardInterrupt: 